#Transfer Learning for Text Summarization with Evaluation (BLEU & ROUGE)

We'll use:

🤗 Hugging Face Transformers for T5 model

Synthetic text-summary pairs

evaluate for BLEU and ROUGE

In [1]:
pip install transformers evaluate nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 1.6 MB/s eta 0:00:00


In [7]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=97d0c5230fa5340dac5e7f055932bc9ec3ca5ef0af7167bf4d217e010b574555
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [2]:
# Import necessary libraries
from transformers import pipeline
import evaluate
import nltk
nltk.download('punkt_tab')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#📝 Step 1: Create Synthetic Data

In [3]:
# Synthetic document-summary pairs
data = [
    {
        "text": "Artificial intelligence is a branch of computer science focused on building smart machines.",
        "summary": "AI is a computer science branch creating smart machines."
    },
    {
        "text": "The capital of France, Paris is known for its fashion, art, and iconic landmarks like the Eiffel Tower.",
        "summary": "Paris is France's capital, famous for art and the Eiffel Tower."
    },
    {
        "text": "Photosynthesis is the process by which green plants use sunlight to make their own food.",
        "summary": "Plants use sunlight in photosynthesis to create food."
    }
]


#🤖 Step 2: Load a Pretrained Model (T5 for Summarization)

In [4]:
# Load summarization pipeline with pre-trained T5 model
summarizer = pipeline("summarization", model="t5-small")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu


#📦 Step 3: Run Summarization on Synthetic Inputs

In [5]:
generated_summaries = []

# Generate summaries for each input
for item in data:
    result = summarizer(item['text'], max_length=30, min_length=5, do_sample=False)
    generated_summary = result[0]['summary_text']
    generated_summaries.append(generated_summary)
    print(f"\nOriginal: {item['text']}")
    print(f"Target Summary: {item['summary']}")
    print(f"Generated Summary: {generated_summary}")


Your max_length is set to 30, but your input_length is only 18. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=9)
Both `max_new_tokens` (=256) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Your max_length is set to 30, but your input_length is only 27. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Both `max_new_tokens` (=256) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Original: Artificial intelligence is a branch of computer science focused on building smart machines.
Target Summary: AI is a computer science branch creating smart machines.
Generated Summary: artificial intelligence is a branch of computer science focused on building smart machines .


Your max_length is set to 30, but your input_length is only 20. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)
Both `max_new_tokens` (=256) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Original: The capital of France, Paris is known for its fashion, art, and iconic landmarks like the Eiffel Tower.
Target Summary: Paris is France's capital, famous for art and the Eiffel Tower.
Generated Summary: the capital of France is known for its fashion, art, and iconic landmarks like the Eiffel Tower .

Original: Photosynthesis is the process by which green plants use sunlight to make their own food.
Target Summary: Plants use sunlight in photosynthesis to create food.
Generated Summary: photosynthesis is the process by which green plants use sunlight to make their own food .


#📊 Step 4: Evaluation using BLEU and ROUGE

In [8]:
# Extract references and predictions
references = [item['summary'] for item in data]
predictions = generated_summaries

# BLEU evaluation
bleu = evaluate.load("bleu")
bleu_score = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
print("\nBLEU Score:", bleu_score)

# ROUGE evaluation
rouge = evaluate.load("rouge")
rouge_score = rouge.compute(predictions=predictions, references=references)
print("\nROUGE Score:")
for k, v in rouge_score.items():
    print(f"{k}: {v:.4f}")



BLEU Score: {'bleu': 0.11119049217415693, 'precisions': [0.48, 0.19148936170212766, 0.06818181818181818, 0.024390243902439025], 'brevity_penalty': 1.0, 'length_ratio': 1.5625, 'translation_length': 50, 'reference_length': 32}

ROUGE Score:
rouge1: 0.5929
rouge2: 0.2376
rougeL: 0.4877
rougeLsum: 0.4877


#💬 Summary
We used transfer learning with T5 for summarization on synthetic data.

Evaluated using:

BLEU (compares n-grams of generated and reference)

ROUGE (recall-oriented metrics like ROUGE-1, ROUGE-L)

This practical showcases how pre-trained models are applied directly to NLP tasks with minimal training.